In [1]:
import os
os.chdir("../") # changi to root directory

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfigEntity:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data_path: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_training_config(self)->TrainingConfigEntity:
        training = self.config.training
        base_model = self.config.base_model
        params = self.params
        training_data_path = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        
        create_directories([Path(training.root_dir)])
        
        training_config = TrainingConfigEntity(
            root_dir=Path(training.root_dir),
            trained_model_path=training.trained_model_path,
            updated_base_model_path=Path(base_model.updated_base_model_path),
            training_data_path= Path(training_data_path),
            params_epochs = params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
            
        )
        
        return training_config

In [5]:
import tensorflow as tf
import time

[2024-02-06 17:14:58,687: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [6]:
class TrainingComponent:
    def __init__(self, config: TrainingConfigEntity):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    def train_valid_generator(self):
        
        dataGen_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
            **dataGen_kwargs
        )
        
        self.valid_generator = valid_dataGen.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
        if(self.config.params_is_augmentation):
            train_dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip = True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **dataGen_kwargs
            )
        else:
            train_dataGen = valid_dataGen
            
        self.train_generator = train_dataGen.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs 
        )
        
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )
        
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [7]:
try:
    config = ConfigurationManager()
    trainig_config = config.get_training_config()
    training = TrainingComponent(config=trainig_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-02-06 17:14:59,687: INFO : common: yaml file: config\config.yaml loaded sucessfully!]
[2024-02-06 17:14:59,691: INFO : common: yaml file: params.yaml loaded sucessfully!]
[2024-02-06 17:14:59,693: INFO : common: Created directory at: artifacts]
[2024-02-06 17:14:59,694: INFO : common: Created directory at: artificats\training]
[2024-02-06 17:14:59,967: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-02-06 17:15:00,086: WARNING : module_wrapper: From c:\Users\samtn\anaconda3\envs\envMlops1_lcc\Lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
[2024-02-06 17:15:00,775: WARNING : optimizer: Skipping variable loading for optimizer 'SGD', because it has 3 variables whereas the saved op

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/training/model.keras'